# Pipelines (Preprocesar y modelar en un solo paso)
* Una habilidad fundamental para implementar modelos complejos con preprocesamiento.
* Pipeline es una forma sencilla de **mantener organizado el código de  preprocesamiento  y modeladode datos**. 
* Específicamente, es una canalización que agrupa pasos de **preprocesamiento y modelado** para que pueda utilizar todo el paquete como si fuera **un solo paso**.

# Construcción del pipeline completo en tres pasos.
## Paso 1: definir los pasos de preprocesamiento
* Pipeline (canalización) agrupa los **pasos** de **preprocesamiento** y **modelado**
* La clase **ColumnTransformer** agrupa diferentes pasos de **preprocesamiento de datos numéricos y categóricos**.
* En el siguiente código:

  * 1. **numerical_transformer**: imputa valores faltantes en **datos numéricos** con **SimpleImputer()**
  * 2. **categorical_transformer** hace dos procesos en **datos categóricos**:
      * **imputa** valores faltantes con **SimpleImputer()**
      * aplica una **codificación con OneHotEncoder()**
  * 3. **preprocessor**: Preprocesamiento de datos numéricos y categóricos con **ColumnTransformer()**

## Paso 2: definir el modelo
* En este  caso, definimos un modelo random forest (bosque aleatorio) con la conocida clase **RandomForestRegressor.**

## Paso 3: crear y evaluar el Pipeline()
* Finalmente, usamos la **clase Pipeline** para definir una canalización que agrupa los pasos de **preprocesamiento** y **modelado**.
* Hay algunas cosas importantes a tener en cuenta:
  * Con el pipeline, **preprocesamos los datos de entrenamiento y ajustamos el modelo en una sola línea de código**.
  * Por el contrario, **sin una canalización, tenemos que realizar la imputación, la codificación one-hot y el entrenamiento del modelo en pasos separados.**
  * ¡Esto se vuelve especialmente complicado si tenemos que lidiar con variables numéricas y categóricas!)
* **Con pipeline**, suministramos al comando predict() las características no procesadas en X_valid y la canalización **preprocesa automáticamente las características antes de generar predicciones**.
* Sin embargo, **sin pipeline**, **debemos preprocesar los datos de validación antes** de hacer predicciones.

# Modelo de Machine Learning para estimar pecios de viviendas

* Se utliza los datos de la Competencia de precios de vivienda para usuarios de Kaggle Learn(https://www.kaggle.com/c/home-data-for-ml-course), donde utilizará 79 variables explicativas diferentes (como el tipo de techo, la cantidad de dormitorios y la cantidad de baños) para predecir los precios de las viviendas.

## Carga de datos
* Cargar los conjuntos de entrenamiento y validación en **X_train, X_valid, y_train e y_valid**.
* El conjunto de prueba se carga en **X_test**.

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Leer los datos
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# 2. Eliminar filas con valores faltantes en la variable objetivo
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)

# 2.1 Separar variable objetivo de las predictoras
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# 3. Separar el conjunto de validación del conjunto de entrenamiento.
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# 4. "Cardinalidad" significa la cantidad de valores únicos en una columna
# Seleccione columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitraria)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]

# 5. Selecciona columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]

# 6. Mantenga solo las columnas seleccionadas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [24]:
# Un vistazo a X_train
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


## Preprocesar los datos y entrenar un modelo con Pipeline

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 1. Preprocesamiento de datos numéricos
numerical_transformer = SimpleImputer(strategy='median')

# 2. Preprocesamiento de datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Preprocesamiento de paquetes para datos numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 4. Definir el modelo de Machine Learning
model = RandomForestRegressor(n_estimators=100, random_state=0)

# 5. Agrupar código de preprocesamiento y modelado en una canalización
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# 6. Preprocesamiento de datos de entrenamiento, modelo de ajuste
clf.fit(X_train, y_train)

# 7. Preprocesamiento de datos de validación, obtención de predicciones
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17553.371061643833


## Preprocesamiento de datos de prueba. X_test

In [31]:
# 8. Preprocesamiento de datos de prueba, modelo de ajuste
preds_test = clf.predict(X_test)
preds_test

array([127226.5 , 154266.5 , 184979.12, ..., 150508.17, 110219.08,
       222463.  ])

In [32]:
# 9. Guardar predicciones en un archivo
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)